<a href="https://colab.research.google.com/github/samarsinghchauhan/Snowlabs/blob/main/DQ_PrototypeGenerator_Snowflake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Rule Id,Rule Description,KDE Table,KDe Column
DQ_TEST_01,Ensures that the customer Birth date  is  less than current date,CUSTOMER,C_BIRTH_DAY
DQ_TEST_02,Ensures that the customer Birth date  is greater than 1900-01-01,CUSTOMER,C_BIRTH_DAY
DQ_TEST_03,Ensures that the customer name contains only alphabets,CUSTOMER,C_NAME
DQ_TEST_04,Ensures that the call center id  contains only numbers,CALL_CENTER,CC_CALL_CENTER_ID

# Task
Generate Python code to read the CSV file "/content/sample_data/input_rules.csv", connect to the Snowflake table "DQ_FRAMEWORK.CONFIG.RULE_METADATA", and for each rule in the CSV, generate a Snowflake query based on the "Rule Description" using the corresponding "Rule Query" and "Population Query" from the Snowflake table as context.

## Set up environment

### Subtask:
Install necessary libraries for Snowflake connection and data handling.


**Reasoning**:
Install the necessary libraries using pip.



In [4]:
%pip install snowflake-connector-python pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.8/73.8 kB 516.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 5.8 MB/s eta 0:00:00


## Load input data

### Subtask:
Read the input CSV file `/content/sample_data/input_rules.csv` into a pandas DataFrame.


**Reasoning**:
The first step of the subtask is to read the CSV file into a pandas DataFrame. I will import pandas and read the CSV, then display the head and info of the DataFrame as requested.



In [41]:
import pandas as pd

df_rules = pd.read_csv('/content/sample_data/input_rules.csv')

display(df_rules.head())
display(df_rules.info())

,Rule Id,Rule Description,KDE Table,KDe Column
0,DQ_TEST_01,Ensures that the customer Birth date is less...,CUSTOMER,C_BIRTH_DAY
1,DQ_TEST_02,Ensures that the customer Birth date is great...,CUSTOMER,C_BIRTH_DAY
2,DQ_TEST_03,Ensures that the customer name contains only a...,CUSTOMER,C_NAME
3,DQ_TEST_04,Ensures that the call center id contains only...,CALL_CENTER,CC_CALL_CENTER_ID


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Rule Id           4 non-null      object
 1   Rule Description  4 non-null      object
 2   KDE Table         4 non-null      object
 3   KDe Column        4 non-null      object
dtypes: object(4)
memory usage: 260.0+ bytes


None

## Connect to snowflake

### Subtask:
Establish a connection to the Snowflake database `DQ_FRAMEWORK`, schema `CONFIG`, and table `CONFIG.RULE_METADATA`.


**Reasoning**:
Establish a connection to the Snowflake database and create a cursor object.



In [42]:
import snowflake.connector
import getpass



conn = snowflake.connector.connect(
     account = 'XTHCUIE-RZC53419',
        user = 'SAMARSINGH01',
        password = 'MySecurePassword2025',
        database = 'DQ_FRAMEWORK',
        schema = 'TPCH_SF1',
        warehouse = 'COMPUTE_WH'
)

cur = conn.cursor()

print("Successfully connected to Snowflake.")

Successfully connected to Snowflake.


## Retrieve rule metadata

### Subtask:
Query the `CONFIG.RULE_METADATA` table to retrieve existing Rule Query and Population Query based on KDE Table and Column from the input CSV.


**Reasoning**:
Iterate through the rows of `df_rules`, construct and execute a Snowflake query to retrieve the `RULE_QUERY` and `POPULATION_QUERY` based on `KDE_TABLE` and `KDE_COLUMN`, and store the results in new columns of `df_rules`.



In [50]:
rule_queries = []
rulemetadata_descriptions = []
population_queries = []

for index, row in df_rules.iterrows():
    kde_table = row['KDE Table']
    kde_column = row['KDe Column']

    query = f"""
        SELECT
            RULE_DESCRIPTION,
            RULE_QUERY,
            POPULATION_QUERY
        FROM
            CONFIG.RULE_METADATA
        WHERE
            KDE_TABLE = '{kde_table}' AND KDE_COLUMN = '{kde_column}';
    """

    cur.execute(query)
    result = cur.fetchone()

    if result:
        rulemetadata_descriptions.append(result[0])
        rule_queries.append(result[1])
        population_queries.append(result[2])
    else:
        rulemetadata_descriptions.append(None)
        rule_queries.append(None)
        population_queries.append(None)

df_rules['Rule Query'] = rule_queries
df_rules['Rule Metadata Descrition'] = rulemetadata_descriptions
df_rules['Population Query'] = population_queries

display(df_rules.head())

,Rule Id,Rule Description,KDE Table,KDe Column,Rule Query,Population Query,Generated Rule ID,Generated Rule Query,Generated Population Query,Rule Descrition,Rule Metadata Descrition
0,DQ_TEST_01,Ensures that the customer Birth date is less...,CUSTOMER,C_BIRTH_DAY,None,None,DQ-001,SELECT COUNT(*) FROM PROEDW.PROSTAGING.CUSTOME...,SELECT COUNT(*) FROM PROEDW.PROSTAGING.CUSTOMER,None,None
1,DQ_TEST_02,Ensures that the customer Birth date is great...,CUSTOMER,C_BIRTH_DAY,None,None,None,None,None,None,None
2,DQ_TEST_03,Ensures that the customer name contains only a...,CUSTOMER,C_NAME,SELECT COUNT(*) AS ERROR_COUNT FROM (select * ...,SELECT COUNT(*) AS POPULATION_COUNT FROM (sele...,None,None,None,Ensures that the customer name is not null for...,Ensures that the customer name is not null for...
3,DQ_TEST_04,Ensures that the call center id contains only...,CALL_CENTER,CC_CALL_CENTER_ID,SELECT COUNT(*) AS ERROR_COUNT FROM PROEPW.PRO...,SELECT COUNT(*) AS POPULATION_COUNT FROM PROEP...,CC_ID_NUMERIC,SELECT COUNT(*) AS ERROR_COUNT FROM PROEPW.PRO...,SELECT COUNT(*) AS POPULATION_COUNT FROM PROEP...,Ensures that the call center id is not null.,Ensures that the call center id is not null.


**Reasoning**:
The previous command failed because the `RULE_METADATA` table was not found or authorized. The connection was established to the database `DQ_FRAMEWORK` and schema `CONFIG`. The error suggests the table `RULE_METADATA` might not be in this schema or the user lacks permissions. To fix this, I will explicitly specify the fully qualified table name `DQ_FRAMEWORK.CONFIG.RULE_METADATA` in the SQL query to ensure it's targeting the correct table within the connected database and schema, and then re-execute the query.



## Generate sql queries

### Subtask:
For each rule in the input data, construct a prompt using the Rule Description and the retrieved Rule Query/Population Query as context. Use a generative model to generate the Snowflake SQL query.


**Reasoning**:
Iterate through the dataframe, construct the prompt for the generative model, and generate the Snowflake SQL query for each rule.



In [58]:
import json

generated_queries = []
generated_rule_ids = []
generated_rule_queries = []
generated_population_queries = []
model='openai-gpt-4.1'
for index, row in df_rules.iterrows():
    rule_id = row['Rule Id']
    rulemetadata_description= row['Rule Metadata Descrition']
    rule_description = row['Rule Description']
    kde_table = row['KDE Table']
    kde_column = row['KDe Column']
    rule_query_context = row['Rule Query']
    population_query_context = row['Population Query']

    # Construct the prompt for the generative model
    prompt = f"""
    You are an expert Data Qualiy Analyst. Based on the following rule description and contextual SQL queries, generate two Snowflake SQL queries that checks for data quality violations.


    Rule Description: {rule_description}
    source database: PROEDW
    Source Schema: PROSTAGING
    Source Table: {kde_table}
    Source Column: {kde_column}
    Contextual Rule Query:
    {rule_query_context}

    Contextual Population Query:
    {population_query_context}

    Generate Snowflake Rule query and SQL Population Query in JSON format with keys "RULE_ID", "RULE_QUERY", and "POPULATION_QUERY"
    Use Snowflake metadata from INFORMATATION_SCHEMA to generate correct quyery
    match the given rule description "{rule_description}" with existing rule description "{rulemetadata_description}":
    """
    print(prompt)

    # Escape single quotes in the prompt for SQL
    escaped_prompt = prompt.replace("'", "''")

    # Call Snowflake Cortex function to generate queries
    query_generated_sql = f"""
    SELECT SNOWFLAKE.CORTEX.COMPLETE(
        'snowflake-arctic',
        '{escaped_prompt}'
    )::string AS generated_sql;
    """

    cur.execute(query_generated_sql)
    result = cur.fetchone()

    if result:
        generated_sql_json = result[0]
        generated_queries.append(generated_sql_json)

        try:
            # Parse the JSON string
            generated_data = json.loads(generated_sql_json)
            generated_rule_ids.append(generated_data.get("RULE_ID"))
            generated_rule_queries.append(generated_data.get("RULE_QUERY"))
            generated_population_queries.append(generated_data.get("POPULATION_QUERY"))
        except json.JSONDecodeError:
            print(f"Error decoding JSON for rule_id: {rule_id}")
            generated_rule_ids.append(None)
            generated_rule_queries.append(None)
            generated_population_queries.append(None)

    else:
        generated_queries.append(None)
        generated_rule_ids.append(None)
        generated_rule_queries.append(None)
        generated_population_queries.append(None)

# Add new columns to the DataFrame to store the parsed data
df_rules['Generated Rule ID'] = generated_rule_ids
df_rules['Generated Rule Query'] = generated_rule_queries
df_rules['Generated Population Query'] = generated_population_queries

# Select the desired columns for the output CSV
output_df = df_rules[['Rule Id', 'Rule Description', 'KDE Table', 'KDe Column', 'Generated Rule ID', 'Generated Rule Query', 'Generated Population Query']]

# Save the output DataFrame to a CSV file
output_df.to_csv('generated_snowflake_queries.csv', index=False)

display(output_df.head())


    You are an expert Data Qualiy Analyst. Based on the following rule description and contextual SQL queries, generate two Snowflake SQL queries that checks for data quality violations.
    
    
    Rule Description: Ensures that the customer Birth date  is  less than current date
    source database: PROEDW
    Source Schema: PROSTAGING
    Source Table: CUSTOMER
    Source Column: C_BIRTH_DAY
    Contextual Rule Query:
    None

    Contextual Population Query:
    None

    Generate Snowflake Rule query and SQL Population Query in JSON format with keys "RULE_ID", "RULE_QUERY", and "POPULATION_QUERY" 
    Use Snowflake metadata from INFORMATATION_SCHEMA to generate correct quyery
    match the given rule description "Ensures that the customer Birth date  is  less than current date" with existing rule description "None":
    

    You are an expert Data Qualiy Analyst. Based on the following rule description and contextual SQL queries, generate two Snowflake SQL queries that checks

,Rule Id,Rule Description,KDE Table,KDe Column,Generated Rule ID,Generated Rule Query,Generated Population Query
0,DQ_TEST_01,Ensures that the customer Birth date is less...,CUSTOMER,C_BIRTH_DAY,DQ_001,SELECT COUNT(*) FROM PROEDW.PROSTAGING.CUSTOME...,SELECT COUNT(*) FROM PROEDW.PROSTAGING.CUSTOMER
1,DQ_TEST_02,Ensures that the customer Birth date is great...,CUSTOMER,C_BIRTH_DAY,CUSTOMER_BIRTH_DATE_CHECK,SELECT COUNT(*) FROM PROEDW.PROSTAGING.CUSTOME...,SELECT COUNT(*) FROM PROEDW.PROSTAGING.CUSTOMER
2,DQ_TEST_03,Ensures that the customer name contains only a...,CUSTOMER,C_NAME,None,None,None
3,DQ_TEST_04,Ensures that the call center id contains only...,CALL_CENTER,CC_CALL_CENTER_ID,CC_ID_NUMERIC,SELECT COUNT(*) AS ERROR_COUNT FROM PROEPW.PRO...,SELECT COUNT(*) AS POPULATION_COUNT FROM PROEP...


**Reasoning**:
The previous code failed because the `generated_sql` variable was not defined when `rule_query_context` or `population_query_context` were None. The code block needs to be fixed to ensure `generated_sql` is always defined before being appended to the list.

